In [1]:
'''
This notebook looks for appropriate rate parameter choices to get good cell population dynamics


basic strategy here

will fix the bath size and adjust just initial cell density, birth rate, and death rate to hit our target.


'''

'\nThis notebook looks for appropriate rate parameter choices to get good cell population dynamics\n\n\nbasic strategy here\n\nwill fix the bath size and adjust just initial cell density, birth rate, and death rate to hit our target.\n\n\n'

In [2]:
#import neccessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# plt.rc('legend',markerscale=20)
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
pd.set_option('display.max_columns', None,'display.max_rows', None)
import pyutilib.services
from importlib import reload

Module_path = 'Modules'  # specify path to modules folder where baf, fopt, and are located, 
                            # change as needed for environment
import sys
sys.path.insert(0,Module_path)
import base_functions as baf
import Bolt_opt_functions as fopt
import analysis_functions as anf
import DynamicThermoOpt_functions_ethan as dyn

In [3]:
#Set initial choice to compute vref
x1_e_init = 1.0e-4
x4_e_init = 1e-7

bath_v = 1.0

In [8]:



model_file = 'Saved_models/smallModel_lowK.pkl'


base_simulation_list=[]
simulation_test={
    'name':'TEST',
    'model':model_file,
    'model_output':'TEST',
#     'secondary_objective': {'protein_syn':1, 'rna_syn':1,'dna_syn':1}, # dictionary 'reaction name': val where val \in reals
#     'primary_obj_tolerance': 100, # positive real, default 100
    'initial_conditions':{}, # dictionary'metabolite name': val where val \in positive reals
    #'boundary_conditions':{}, # dictionary'metabolite name': val where val \in positive reals
    #'boundary_conditions':metab_shift,
    'boundary_conditions':{'B:CYTOSOL':1e-8,'x1_e:ENVIRONMENT':x1_e_init,'x4_e:ENVIRONMENT':x4_e_init},
    #'obj_coefs':{'rna_syn': 1.0,'protein_syn': 1.0, 'dna_syn':1.0}, # dictionary'reaction name': val where val \in reals; if this is defined 
#     'remove_metabolites':['SUCROSE:ENVIRONMENT'], # list'metabolite name'
#     'remove_reactions':['RXN-1461'], # list 'reaction name'
    'implicit_metabolites':[], # list 'metabolite name', default for Fixed species
    'explicit_metabolites':[], # list 'metabolite name', default for Variable species
#     'redox_pairs':[
#        ['NADPH:CYTOSOL' ,'NADP+:CYTOSOL'],
#        ['A REDUCED THIOREDOXIN:CYTOSOL', 'AN OXIDIZED THIOREDOXIN:CYTOSOL'],
#        ['UBIQUINOL:MITOCHONDRIAL_MEM','UBIQUINONE:MITOCHONDRIAL_MEM'],
#         ['OXYGEN:CYTOSOL' ,'GMP:CYTOSOL']
#     ], # list of pairs of metabolite names [[a,b],[c,d]] if both variable then fix first name (i.e a and c)
#     'redox_ratios':[{'names':['NADH:CYTOSOL','NAD+:CYTOSOL'],'ratio':1.2},
#                    {'names':['NADH:MITOCHONDRIA','NAD+:MITOCHONDRIA'],'ratio':1.2}], # list of dictionaries containing a list of pair of metabolites and the ratio
#     'raise_bounds':['PROTEIN','RNA','DNA'],
#     'specify_upper_bound':{'ALPHA-D-GLUCOSE:CYTOSOL':1e-3}, # default 1e-3
#     'specify_lower_bound':{'ALPHA-D-GLUCOSE:CYTOSOL':1e-120}, # default 1e-120
    'net_flux':{}, # dictionary 'metabolite name': val where val \in {1, 0, -1}, default 0 
    'flux_direction':{}, # dictionary 'reaction_name': val where val \in {1, 0, -1}, default 0 
    'allow_regulation':{} # dictionary 'reaction_name': val where val \in {1, 0}, default 1 
}
base_simulation_list.append(simulation_test)




control_dictionary={
    'model_file_dir':'Saved_models', # default 'Saved_models'
    'solution_file_dir':'Saved_solutions', # default 'Saved_solutions'
    'parallel_solve':False,# boolean True/False
    'worker_count': 1,  # natural number, 0 or remove for cpu_count
    'delta_concentration':[1e-4], # list positive reals
    'delta_flux':[1], # list positive reals
    'mb':[100], # list positive naturals
    'zeta':[0.001], # list positive reals
    'swarm_size':30, # naturals, default is 2
    'linear_solver': 'ma57',
    'max_cpu_time':8000000,
    'max_iter':10000,
    'acceptable_tol':1e-6,
    'solver_tol':1e-7,
    'feasibility_check':False,
    'feasibility_as_input':False,
    'annealing_check':False,
    'hsllib':'/opt/coinhsl/lib/libcoinhsl.dylib'
}




cell_dictionary={
    'model':'smallModel_lowK.pkl',#'smallModel_highK.pkl',
    'previous_best_model_solution':"",
    'environment_energy_name':'x1_e:ENVIRONMENT',
    'energy_import_reaction_name':'Uptake',
    'environment_compartment_name':'ENVIRONMENT',
    
#     'adaptation_constant':0.01,
    
    'initial_cell_density':1e5,
    'birth_rate':2e7,
    'death_rate':0.001,
    
    'import_energy_metabolites':['x1_e:ENVIRONMENT','x4_e:ENVIRONMENT'],
    'import_energy_reactions':['Uptake','Excretion'],
    'import_energy_michaelis_menten':[0.04*1e-3,0.04*1e-3], # m moles
    
    'vmax':3.033e-12, # mM glucose consumed per hour per 1 micron^3 cell 
    'cell_volume':1e-15, 
    'hyphae_volume':4e3, # microns^3
    'N_avogadro':6.022140857e+23
    
}



ode_system_dictionary={
    'dt':1.0,
    'nmax':7100,
    # 'nmax':1000000,
#     'initial_concentration_perturbation':0.0,
    'stoichiometry_matrix':[],
    'metabolites_to_fix':['B:CYTOSOL'],
    'jump_tolerance':0.5,
    'stall_on_opt_jump':5,
    'Opt_frequency':100000
#     'Opt_frequency':100000 # after how many timesteps to recompute kinetic rates through OPT.
#     'metabolites_to_fix':['x1_e:ENVIRONMENT','x4_e:ENVIRONMENT','B:CYTOSOL']
}

environment_dictionary={
    'metabolites':['x1_e:ENVIRONMENT','x4_e:ENVIRONMENT'],
    'transport_reactions':['Uptake','Excretion'],
#     'diffusion_rates':[600e-12,600e-12],  # µm^2/sec
#     'boundary_concentrations':[1e-4,1e-6],
# for newmann BCs +ve value for influx int environment from external resevoir
#     'chemostat_type':[['Zero',{'Magnitude':1e-4}],['Zero',{'Magnitude':1e-4}]],
    'chemostat_type':[['Zero',{'Magnitude':1e-4}],['Zero',{'Magnitude':1e-6}]],
    'initial_environment_concentrations':[x1_e_init,x4_e_init], # mili-molar
    
    'bath_volume':bath_v,
    
    'glucose_diffusion_constant':600 # µm^2/sec
}





''' 
RUN OPTIMIZATION
'''
t_opt = 0
i = 0
### Some setup tasks
chemostatDf,environmentDf,cellContributionDf = dyn.SetUpEnvironmentCellDataFrames(environment_dictionary,ode_system_dictionary,False)
modelBoundaryConditions = dyn.SetInitialModelBoundary(i,environment_dictionary,cell_dictionary,ode_system_dictionary,environmentDf,chemostatDf,cellContributionDf)
cell_dictionary = dyn.SetUpScaling(cell_dictionary,environment_dictionary,ode_system_dictionary,True)
ode_system_dictionary['initialiseODE']=True

### optimization computation
base_simulation_list = dyn.ComposeBaseSimulation(modelBoundaryConditions,t_opt,cell_dictionary)
bestSimulations = baf.RunSwarmSimulation(base_simulation_list,control_dictionary)
metabolite_concs_opt,flux_opt,S_opt,active_reactions_opt = dyn.ParseBestSimulations(bestSimulations,control_dictionary,cell_dictionary)#,[],[],True)


metaboliteTraceDf,fluxTraceDf,simKineticsDf,k_app_f,k_app_r,sim_concs = dyn.SetupODESystem(ode_system_dictionary,environment_dictionary,metabolite_concs_opt,flux_opt)
v_hat,a_hat,n_hat,S,Keq,metabolite_labels,reaction_labels = dyn.ReadOPTSolution(bestSimulations,control_dictionary,cell_dictionary)
cell_dictionary = dyn.InitializeKineticConversion(t_opt,v_hat,S,Keq,n_hat,cell_dictionary,environment_dictionary,ode_system_dictionary,metabolite_labels,reaction_labels)



''' 
CONVERSIONS FOR ODE SIMULATION
'''
v_hat,a_hat,n_hat,S,Keq,metabolite_labels,reaction_labels = dyn.ReadOPTSolution(bestSimulations,control_dictionary,cell_dictionary)
## first we need to convert fluxes into our ode time scale
kineticsDf = dyn.DetermineOptimalKinetics(t_opt,v_hat,S,Keq,n_hat,cell_dictionary,environment_dictionary,ode_system_dictionary,metabolite_labels,reaction_labels)


''' 
COMPUTE FLUXES THAT INTERACT WITH ENV AND CELL POP
'''


#first we need our relevant variables
biomass_flux = kineticsDf['Optimal Flux'].loc['Biomass']
uptake_flux = kineticsDf['Optimal Flux'].loc['Uptake']
excretion_flux = kineticsDf['Optimal Flux'].loc['Excretion']


Sim Name: ChemostatModel_t_0_swarm_0_100_0.001_0.0001_1 objective: 116.12905298770902 status: ok
Sim Name: ChemostatModel_t_0_swarm_1_100_0.001_0.0001_1 objective: 220.62495132390856 status: ok
Sim Name: ChemostatModel_t_0_swarm_2_100_0.001_0.0001_1 objective: 220.62495132390794 status: ok
Sim Name: ChemostatModel_t_0_swarm_3_100_0.001_0.0001_1 objective: 116.12905300928907 status: ok
Sim Name: ChemostatModel_t_0_swarm_4_100_0.001_0.0001_1 objective: 116.12905299990693 status: ok
Sim Name: ChemostatModel_t_0_swarm_5_100_0.001_0.0001_1 objective: 220.6249513239071 status: ok
Sim Name: ChemostatModel_t_0_swarm_6_100_0.001_0.0001_1 objective: 116.1290529877083 status: ok
Sim Name: ChemostatModel_t_0_swarm_7_100_0.001_0.0001_1 objective: 116.12905298770877 status: ok
Sim Name: ChemostatModel_t_0_swarm_8_100_0.001_0.0001_1 objective: 220.62495132393238 status: ok
Sim Name: ChemostatModel_t_0_swarm_9_100_0.001_0.0001_1 objective: 220.62495132390575 status: ok
Sim Name: ChemostatModel_t_0_swa

In [9]:
biomass_flux

2.168618921241087

In [10]:
uptake_flux

2.1664285714285714

In [13]:
#### Some rate parameterization


'''
Lets start by saying that we have
1 Liter of volume to work with

we have 2e12 cells per 1 gram dry weight

gram_cell = 0.5e-13

so we have cell_grams_mol approx 3.01e10 


'''

###PARMS
gluc_gram_mole = 180.156
cell_gram_mole = 3.01e10
avgadro_num = 6.02214076e23
cell_volume = cell_dictionary['cell_volume']

bath_volume = 1 #Liters



glucose_grams = 0.1
glucose_moles = glucose_grams/gluc_gram_mole
gluc_conc_mol_L = (glucose_moles)/(bath_volume)
print('glucose concentration mol per liter: ', gluc_conc_mol_L)



cell_grams = 0.01 #grams
cell_moles = cell_grams/cell_gram_mole
cell_conc_nanomol_L = (cell_moles*1e9)/(bath_volume)
print('cell concentration nanomol per Liter: ', cell_conc_nanomol_L)




#How fast glucose concentrations are changing
print('glucose uptake Mol per L (cell volume) per hour in each cell: ', uptake_flux)
uptake_gluc_Mol_L_bath = uptake_flux*cell_volume*cell_conc_nanomol_L*(avgadro_num / 1e9)
print('glucose uptake Mol per L (bath volume) per hour by all cells: ', uptake_gluc_Mol_L_bath)



#Cell Death rate
cell_death_rate = 0.008  #per hour
#convert to moles/liter
cell_death_rate_nanomol_L = cell_death_rate*1e9/(avgadro_num*bath_volume)
print('per cell death rate: ', cell_death_rate)
print('cell death nanomol per L per hour: ', cell_death_rate* cell_conc_nanomol_L)



'''  
Cell Growth Rate
if we halve the biomass rate we should have roughly the glucose uptake rate
we know that every gram of glucose in contributes to .3 grams of cell mass.
we need to convert grams of cell mass to nanomoles of cells
biomass_flux * cell_volume * 1/3 * gluc_gram_mole * (1/cell_gram_mole) * 1e9
'''
gramBiomass_to_cellcount =  5e-13
grams_of_biomass = biomass_flux*cell_volume*(0.3)*gluc_gram_mole
per_cell_growth_rate  = grams_of_biomass/gramBiomass_to_cellcount
birth_rate_nanomol_L = per_cell_growth_rate*1e9/(avgadro_num*bath_volume)

print('per cell growth rate: ', per_cell_growth_rate)
print('growth nanomol per L per hour: ', per_cell_growth_rate*cell_conc_nanomol_L)


#quantity_biomass_flux = biomass_flux(milimoles/liter)*volume_cell

#compute rates
#cell_growth_rate = biomass_flux *cell_density*growth_rate
#cell_death_rate = cell_density*death_rate
#uptake_rate = uptake_flux*cell_density*bath_v
#excretion_rate = excretion_flux*cell_density*bath_v


glucose concentration mol per liter:  0.0005550744909966918
cell concentration nanomol per Liter:  0.0003322259136212625
glucose uptake Mol per L (cell volume) per hour in each cell:  2.1664285714285714
glucose uptake Mol per L (bath volume) per hour by all cells:  0.0004334397941404842
per cell death rate:  0.008
cell death nanomol per L per hour:  2.6578073089700997e-06
per cell growth rate:  0.2344138262250656
growth nanomol per L per hour:  7.787834758307827e-05
